# Pipeline - School Roads - Schools

The purpose of this notebook is to gather relative locations of schools in a standardized format for later use in a pipeline. These points provide a general location of schools within a particular region. Because this data may come from various sources, the assumption of this notebook is that each row of data provided has a column indicating the latitude and longitude. This point is a relative location for a larger "polygon" that represents the space by which roads may be near.

The data collected by this notebook will later be used to determine whether specific segments of roads are near a school and if they are, to further analyze whether those roads have street-level signage indicating and warning drivers about the school zone. This data in turn is used to help automatically determine the iRAP 5-star attribute code for school signage.

## Imported Data

The assumption is that incoming data will include attributes found in the "parameters" cell below.

## Exported Data

Data will be in parquet format and include at least columns for "lat" (latitude) and "lon" (longitude)

In [1]:
import os
import zipfile

import pandas as pd
import requests

In [2]:
# Parameters cell used to indicate parameters which will be used at runtime.
# Note: the below is a default parameter value which is overridden when the
# notebook is executed as part of a pipeline via Prefect + Papermill

name = "usa"
link = "https://nces.ed.gov/programs/edge/data/EDGE_GEOCODE_PUBLICSCH_1819.zip"
unzip = True
target = "EDGE_GEOCODE_PUBLICSCH_1819.xlsx"
lat_colname = "LAT"
lon_colname = "LON"

In [3]:
# create a data subdir if it doesn't exist
if not os.path.isdir("{}/data".format(os.getcwd())):
    os.mkdir("{}/data".format(os.getcwd()))

In [4]:
# download file
if not os.path.isfile("{}/data/{}".format(os.getcwd(), os.path.basename(target))):
    getfile = requests.get(link)
    if getfile.status_code == 200:
        with open(
            "{}/data/{}".format(os.getcwd(), os.path.basename(link)), "wb"
        ) as f:
            f.write(getfile.content)

In [5]:
# unzip if necessary
if unzip and not os.path.isfile(
    "{}/data/{}".format(os.getcwd(), os.path.basename(target))
):
    with zipfile.ZipFile(
        "{}/data/{}".format(os.getcwd(), os.path.basename(link)), "r"
    ) as zip_ref:
        zip_ref.extractall("{}/data".format(os.getcwd()))

In [6]:
# read target file into dataframe
if os.path.splitext(target)[1] == ".xlsx":
    df = pd.read_excel("{}/data/{}".format(os.getcwd(), target))
elif os.path.splitext(target)[1] == ".csv":
    df = pd.read_csv("{}/data/{}".format(os.getcwd(), target))

In [7]:
# rename the columns to standardize the pipeline
df = df.rename(columns={lat_colname: "lat", lon_colname: "lon"})

In [15]:
# target individual city, top 5 results for testing purposes
df = df[df["CITY"] == "Chicago"].head(5)

In [16]:
# export to parquet
df.to_parquet("{}/data/{}_schools.parquet".format(os.getcwd(), name), index=False)